In [195]:
import tensorflow as tf
import numpy as np

In [196]:
# Get Data
g=open('sentiment-network/reviews.txt','r')
text=g.read()

In [197]:
#Remove punctuation
from string import punctuation
text=[eachchar for eachchar in text if eachchar not in punctuation]
text=''.join(text)

In [198]:
allwords=text.replace('\n',' ')
reviews=text.split('\n')
reviews=[review.split(' ') for review in reviews]

In [199]:
# Convert words to integers
def word_to_int(text):
    uniqwords=set(text.split(' '))
    vocab_to_int={word:index for index,word in enumerate(uniqwords)}
    int_to_vocab={index:word for index,word in enumerate(uniqwords)}
    return vocab_to_int,int_to_vocab
vocab_to_int,int_to_vocab=word_to_int(allwords)

In [200]:
reviews_ints=[]
for review in reviews:
    reviews_ints.append([vocab_to_int[word] for word in review])
    
reviews_ints=np.array([review[:50] for review in reviews_ints if len(review) >1 ])

In [201]:
labeldata=open('sentiment-network/labels.txt','r')

In [202]:
labeldata=labeldata.read()
labeldata=labeldata.split('\n')
labeldata=[1 if label=='positive' else 0 for label in labeldata]
labeldata=np.array(labeldata[:25000])


In [203]:
def get_batches(x,y,batchsize=100):
    num_batches=len(x)//batchsize
    x=x[:batchsize*num_batches]
    y=y[:batchsize*num_batches]
    for i in range(0,len(x),batchsize):
        yield x[i:i+batchsize],y[i:i+batchsize]

In [205]:
# Build Graph
graph=tf.Graph()
with graph.as_default():
    #REVIEW why shape is None,None
    inputs=tf.placeholder(tf.int32,shape=[None,None],name='inputs')
    labels=tf.placeholder(tf.int32,shape=[None,None],name='labels')
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    
#Embedding
    embed_size=300
    embedding=tf.Variable(expected_shape=tf.random_uniform(len(vocab_to_int),embed_size))
    embed=tf.nn.embedding_lookup(embedding, inputs)